In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#드라이브에 접근할 수 있도록 아래 코드 입력
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#불러올 파일의 경로를 filename 변수에 저장
filename = '/content/drive/MyDrive/AI부트캠프/프로젝트/프로젝트6/쿠팡조회결과.csv'

#pandas read_csv로 불러오기
df = pd.read_csv(filename)
df.head()

In [ ]:
pip install konlpy

In [ ]:
from konlpy.tag import Okt, Kkma, Hannanum
import re

df_name = df['name']
df_name

In [ ]:
#전처리

# 정규식
# []: [] 사이 문자를 매치, ^: not
regex = r"[^가-힣]"


# 치환할 문자
subst = ""


def clean_text(texts):
    corpus = [] 
    for i in range(0, len(texts)): 
        text = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation 
        text = re.sub(r'\d+','', str(texts[i]))# remove number 
        text = text.lower() #lower case 
        text = re.sub(r'\s+', ' ', text) #remove extra space 
        text = re.sub(r'<[^>]+>','',text) #remove Html tags 
        text = re.sub(r'\s+', ' ', text) #remove spaces 
        text = re.sub(r"^\s+", '', text) #remove space from start 
        text = re.sub(r'\s+$', '', text) #remove space from the end
        text = re.compile(regex).sub(subst, text) #한글만
        corpus.append(text) 
    return corpus

df['clean_name'] = clean_text(df_name)

In [ ]:
print(clean_text(df_name))

In [ ]:
test = '프로게이너아몬드프로틴비건인증식물성프로틴비건식단채식식단단백질쉐이크'

def tokenize(text):
    """text 문자열을 의미있는 단어 단위로 list에 저장합니다.
    Args:
        text (str): 토큰화 할 문자열
    Returns:
        list: 토큰이 저장된 리스트
    """
    okt = Okt() 

    tokens = okt.morphs(text) #morphs: 형태소 단위로 구문을 분석
    
    return tokens

print(tokenize(test))

In [ ]:
Han = Hannanum()

print(Han.analyze(test))

In [ ]:
df['tokens'] = df['clean_name'].apply(tokenize)
df['tokens'].head()

In [ ]:
Kkma = Kkma()

print(Kkma.morphs(test))

In [ ]:
#불용어 처리
stopwords = ['개', '개입', '입'] 
def remove_stopword_text(text): 
    corpus = [] 
    for sent in text: 
        modi_sent = [] 
        for word in sent: 
            if word not in stopwords: 
                modi_sent.append(word) 
        corpus.append(''.join(modi_sent)) 
    return corpus

df['tokens'] = df['tokens'].apply(remove_stopword_text)
df['tokens'].head()

In [ ]:
df['tokens'] = df['tokens'].apply(lambda x : (' ').join(x))

df['tokens']

In [ ]:
df

In [ ]:
df_name = df_name.apply(remove_stopword_text)
df_name

In [ ]:
df['clean_name'] = df['clean_name'].apply(lambda x : (' ').join(x))
df['clean_name']

In [ ]:
# # replace 함수를 이용해서 공백 제거 
# df['clean_name'] = df['clean_name'].replace(" " , "")

In [ ]:
df

In [ ]:
coupangdata = df.loc[:, [col for col in df.columns if col != 'clean_name']]

In [ ]:
coupangdata

In [ ]:
coupangdata.to_csv('coupangdata.csv', index=False)

In [ ]:
# tokens에 대해서 tf-idf 수행

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['tokens'])
print(tfidf_matrix.shape)

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
print(cosine_sim)

In [ ]:
indices = pd.Series(df.index, index=df['tokens']).drop_duplicates()
print(indices.head())

In [ ]:
idx = indices['프로 게 이 너 아몬드 프로 틴 비건 인증 식물성 프로 틴 비건 식단 채식 식단 단백질 쉐이크']
print(idx)

In [ ]:
sim_scores = list(enumerate(cosine_sim[4]))

In [ ]:
print(sim_scores)

In [ ]:
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

In [ ]:
print(sim_scores)

In [ ]:
sim_scores = sim_scores[1:4]

In [ ]:
print(sim_scores)

In [ ]:
name_indices = [i[0] for i in sim_scores]

In [ ]:
def get_recommendations(name, cosine_sim=cosine_sim):
    # 선택한 상품의 이름으로부터 해당되는 인덱스를 받아온다.
    idx = indices[name]

    # 모든 상품에 대해서 해당 상품와의 유사도를 구한다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 상품들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 3개의 상품이름을 받아온다.
    sim_scores = sim_scores[1:6]

    # 가장 유사한 3개의 상품이름의 인덱스를 얻는다.
    name_indices = [i[0] for i in sim_scores]

    # 가장 유사한 3개의 상품 이름을 리턴한다.
    return df['tokens'].iloc[name_indices]

In [ ]:
#입력데이터 값이 파일에 있는 것과 정확히 동일해야만 작동
get_recommendations('프로 게 이 너 아몬드 프로 틴 비건 인증 식물성 프로 틴 비건 식단 채식 식단 단백질 쉐이크')

In [ ]:
from collections import Counter

# Counter 객체는 리스트요소의 값과 요소의 갯수를 카운트 하여 저장하고 있습니다.
# 카운터 객체는 .update 메소드로 계속 업데이트 가능합니다.
word_counts = Counter()

# 토큰화된 각 리뷰 리스트를 카운터 객체에 업데이트 합니다. 
df['tokens'].apply(lambda x: word_counts.update(x))

# 가장 많이 존재하는 단어 순으로 10개를 나열합니다
word_counts.most_common(20)